###  Cluster Creation and Evaluation 

In [ ]:
from sklearn.model_selection import train_test_split

df1=keys_df.loc[:,['key', 'score', 'LLM_embedding', 'key_umap']]


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from math import sqrt
import prince
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import umap.umap_ as umap
import numpy as np

In [ ]:
#Intialize clustering and partitioning
kmeans = KMeans(n_clusters=6,init='k-means++', max_iter=300, n_init=10) 
cv = KFold(n_splits=6, random_state=42, shuffle=True)

counter=0
#Split data into 5 or 10 partitions
for train_index, test_index in cv.split(df1):
    X_train, X_test = df1.iloc[train_index], df1.iloc[test_index]
    
    # Convert strings representing lists to actual lists of floats (if required)
    X_train["key_umap"] = X_train["key_umap"].apply(lambda x: [float(val) for val in x.strip('][').split(', ')])
    X_test["key_umap"] = X_test["key_umap"].apply(lambda x: [float(val) for val in x.strip('][').split(', ')])

    #print(X_train.shape)
    # Take only the scores/features for both testing and training
    X_train= X_train[['key', 'score', 'ada_embedding', 'key_umap']]
    X_test= X_test[['key', 'score', 'ada_embedding', 'key_umap']]
    
    # Convert to NumPy array
    embedding_values_array = np.array(X_train["key_umap"].tolist())
    
    # Reduce dimensionality using UMAP
    umap_embedding = umap.UMAP().fit_transform(embedding_values_array)
   
    # Apply KMeans clustering with UMAP
    #kmeans = KMeans(n_clusters=5, random_state=0)  # You may adjust the number of clusters as needed
    labels = kmeans.fit_predict(umap_embedding)
    X_train["label"] = [str(label) for label in labels]
    #print(f"Num of clusters: {len(np.unique(labels))}")
    
    #Model fit with Testing data
    embedding_values_array_test = np.array(X_test["key_umap"].tolist())
    # Reduce dimensionality using UMAP
    umap_embedding_test = umap.UMAP().fit_transform(embedding_values_array_test)
    y_pred_test=kmeans.predict(umap_embedding_test)
    X_test["label"] = [str(label) for label in y_pred_test]
    #print(f"Num of clusters: {len(np.unique(labels))}")  

    # Calculate the sum of values in "key_umap" for each row
    X_train['sum_umap_train'] = X_train['key_umap'].apply(np.sum)
    X_test['sum_umap_train'] = X_test['key_umap'].apply(np.sum)
    
    # incude the sum of key_umap as new column
    Xtrain=X_train[['score','label', 'sum_umap_train']]
    Xtest=X_test[['score','label', 'sum_umap_train']]

    #Train :  group and sum values of each cluster on train
    Xtrain=Xtrain.groupby(['label']).sum()
    Xtrain2=Xtrain.T
    Xtrain2.columns=['c0', 'c1', 'c2', 'c3', 'c4','c5'];
    #Xtrain2.columns=['c0', 'c1', 'c2'];
    Xtrain3=Xtrain2.T
    
    # Test: group and sum values of each cluster  on test
    Xtest=Xtest.groupby(['label']).sum()
    Xtest2=Xtest.T
    Xtest2.columns=['c0', 'c1', 'c2','c3', 'c4','c5'];
    Xtest3=Xtest2.T

    #Total sum of clusters on train
    Xtrain3_sums = Xtrain3.sum(axis=0)
    
    # Train: Normalize values by dividing the values of each cluster by the total number 
    ct0=pd.Series(Xtrain3.iloc[0]/Xtrain3_sums)
    ct1=pd.Series(Xtrain3.iloc[1]/Xtrain3_sums)
    ct2=pd.Series(Xtrain3.iloc[2]/Xtrain3_sums)
    ct3=pd.Series(Xtrain3.iloc[3]/Xtrain3_sums)
    ct4=pd.Series(Xtrain3.iloc[4]/Xtrain3_sums)
    ct5=pd.Series(Xtrain3.iloc[5]/Xtrain3_sums)
 

    #Join clusters as columns and rename
    df_per_train=pd.concat([ct0,ct1,ct2,ct3,ct4,ct5],axis=1)
    df_per_train.columns = ['c0', 'c1', 'c2','c3', 'c4','c5']
    #df_per_test=df_per_train.T
    df_per_train

    #Total sum of clusters on Test
    Xtest3_sums = Xtest3.sum(axis=0)
    
    # # Test: Normalize values by dividing the values of each cluster by the total number 
    c00=pd.Series(Xtest3.iloc[0]/Xtest3_sums)
    c10=pd.Series(Xtest3.iloc[1]/Xtest3_sums)
    c20=pd.Series(Xtest3.iloc[2]/Xtest3_sums)
    c30=pd.Series(Xtest3.iloc[3]/Xtest3_sums)
    c40=pd.Series(Xtest3.iloc[4]/Xtest3_sums)
    c50=pd.Series(Xtest3.iloc[5]/Xtest3_sums)

    #Join clusters as columns and rename
    df_per_test=pd.concat([c00,c10,c20,c30,c40,c50],axis=1)
    df_per_test.columns = ['c0', 'c1', 'c2','c3','c4','c5']
    #df_per_test=df_per_train.T
    df_per_test

    # Define overall RMSE variable before the loop
    overall_rmse_per_cluster = []

    # Calculate RMSE for each partition/sample data
    
    for i in range(6):
        #train_df = train_dfs[i]
        #test_df = test_dfs[i]
        
        # Calculate the squared errors for each cluster in each partition
        squared_errors = (df_per_train.values) - (df_per_test.values) ** 2
        # Calculate the mean squared error for each cluster in each partition
        mean_squared_error = np.mean(squared_errors)
        # Calculate the root mean squared error (RMSE) for each cluster in each partition
        rmse_per_cluster = np.sqrt(mean_squared_error)
        #print(f"RMSE for partition {i+1} for each cluster:", rmse_per_cluster)
        # Append the RMSE for this partition to the overall RMSE list
        overall_rmse_per_cluster.append(rmse_per_cluster)
        
    print("Sample:",counter)
    print("Training set:",X_train.shape)
    print(f"Num of clusters: {len(np.unique(labels))}") 
    counter += 1
        
    # Convert the overall RMSE list to a numpy array
    overall_rmse_per_cluster = np.array(overall_rmse_per_cluster)
    
    # Calculate the overall RMSE by averaging the RMSEs of all clusters across all partitions
    overall_rmse = np.mean(overall_rmse_per_cluster)
    
    #print("\nOverall RMSE for each cluster:", [round(val, 4) for val in overall_rmse_per_cluster])
    print("\nOverall RMSE across all clusters and partitions:", round(overall_rmse, 4))